# Lab 3: Using MemGPT to build agents with memory 
This lab will go over: 
1. Creating an agent with MemGPT
2. Understand MemGPT agent state (messages, memories, tools)
3. Understanding core and archival memory
4. Building agentic RAG with MemGPT 

## Setup a Letta client 
Make sure you run `pip install letta` and `letta quickstart`

In [ ]:
!pip install letta
! letta quickstart

In [ ]:
from letta import create_client 

client = create_client() 

In [ ]:
from letta import LLMConfig, EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini")) 
client.set_default_embedding_config(EmbeddingConfig.default_config(provider="openai")) 

## Creating a simple agent with memory 
MemGPT allows you to create persistent LLM agents that have memory. By default, MemGPT saves all state related to agents in a database, so you can also re-load an existing agent with its prior state. We'll show you in this section how to create a MemGPT agent and to understand what memories it's storing. 


### Creating an agent 

In [ ]:
agent_name = "simple_agent"

In [ ]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)
response

Note that MemGPT agents will generate an *internal_monologue* that explains its actions. You can use this monoloque to understand why agents are behaving as they are. 

Second, MemGPT agents also use tools to communicate, so messages are sent back by calling  a `send_message` tool. This makes it easy to allow agent to communicate over different mediums (e.g. text), and also allows the agent to distinguish betweeh that is and isn't send to the end user. 

### Understanding agent state 
MemGPT agents are *stateful* and are defined by: 
* The system prompt defining the agent's behavior (read-only)
* The set of *tools* they have access to 
* Their memory (core, archival, & recall)

In [ ]:
print(agent_state.system)

In [ ]:
agent_state.tools

### Viewing an agent's memory

In [ ]:
memory = client.get_core_memory(agent_state.id)

In [ ]:
memory

In [ ]:
client.get_archival_memory_summary(agent_state.id)

In [ ]:
client.get_recall_memory_summary(agent_state.id)

In [ ]:
client.get_messages(agent_state.id)

## Understanding core memory 
Core memory is memory that is stored *in-context* - so every LLM call, core memory is included. What's unique about MemGPT is that this core memory is editable via tools by the agent itself. Lets see how the agent can adapt its memory to new information.

### Memories about the human 
The `human` section of `ChatMemory` is used to remember information about the human in the conversation. As the agent learns new information about the human, it can update this part of memory to improve personalization. 

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Bob", 
    role = "user"
) 
response

In [ ]:
client.get_core_memory(agent_state.id)

### Memories about the agent
The agent also records information about itself and how it behaves in the `persona` section of memory. This is important for ensuring a consistent persona over time (e.g. not making inconsistent claims, such as liking ice cream one day and hating it another). Unlike the `system_prompt`, the `persona` is editable - this means that it can be used to incoporate feedback to learn and improve its persona over time. 

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate", 
    role = "user"
) 
response

In [ ]:
client.get_core_memory(agent_state.id).get_block('persona')

## Understanding archival memory
MemGPT agents store long term memories in *archival memory*, which persists data into an external database. This allows agents additional space to write information outside of its context window (e.g. with core memory), which is limited in size. 

In [ ]:
client.get_archival_memory(agent_state.id)

In [ ]:
client.get_archival_memory_summary(agent_state.id)

Agents themselves can write to their archival memory when they learn information they think should be placed in long term storage. You can also directly suggest that the agent store information in archival. 

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'bob loves cats' to archival", 
    role = "user"
) 
response

In [ ]:
client.get_archival_memory(agent_state.id)[0].text

You can also directly insert into archival memory from the client. 

In [ ]:
client.insert_archival_memory(
    agent_state.id, 
    "Bob's loves boston terriers"
)

Now lets see how the agent uses its archival memory:

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
response